In [61]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
from tqdm import tqdm_notebook
import concurrent.futures
from multiprocessing import Pool
import copy,os,sys,psutil
from collections import Counter

In [120]:
import numpy as np

# PyTorch AllenNLP
AllenNLP 的 pyTorch 版本需要配置文件，暂时没有搞清楚配置文件怎么写

In [1]:
from allennlp.modules.elmo import Elmo, batch_to_ids

In [2]:
options_file = "options.json"  # 配置文件地址 
weight_file = "weights.hdf5" # 权重文件地址
# 这里的1表示产生一组线性加权的词向量。
# 如果改成2 即产生两组不同的线性加权的词向量。
elmo = Elmo(options_file, weight_file, 1, dropout=0)
# use batch_to_ids to convert sentences to character ids
sentence_lists = ["I have a dog", "How are you , today is Monday","I am fine thanks"]
character_ids = batch_to_ids(sentences_lists)
embeddings = elmo(character_ids)['elmo_representations']

FileNotFoundError: file options.json not found

# Tensorflow_hub AllenNLP
- [Introduction to TensorflowHub: Simple Text Embedding(Using ELMo)](https://medium.com/@joeyism/embedding-with-tensorflow-hub-in-a-simple-way-using-elmo-d1bfe0ada45c)
- [Tensorflow+Keras做ELMo（中文翻译）](https://ai.yanxishe.com/page/TextTranslation/1597)

In [6]:
import tensorflow as tf
import tensorflow_hub as hub
elmo = hub.Module("https://tfhub.dev/google/elmo/2")

## 参考自[TensorflowHub的官网](https://tfhub.dev/google/elmo/2)

### 输入
#### 参数
- `signature`
    - default | 默认是分句子输入，batch_size等于句子的个数
        > the module takes untokenized sentences as input. The input tensor is a string tensor with shape [batch_size]. The module tokenizes each string by splitting on spaces.
    - tokens  | 使用分过词的句子作为输入
        > With the tokens signature, the module takes tokenized sentences as input. The input tensor is a string tensor with shape [batch_size, max_length] and an int32 tensor with shape [batch_size] corresponding to the sentence length. The length input is necessary to exclude padding in the case of sentences with varying length.
    - 上述两种方式的结果是完全一致的
- `as_dict`
    - 不提供参数 `as_dict`时（默认为 `False`），会使用`default`。即 `elmo(tokens, as_dict=True)['default']` = `elmo(tokens)`

In [131]:
# 使用 signature = default， 输入是句子
sentences = ["the cat is on the mat", "dogs are in the fog"]
emb_opt_sentence = elmo(inputs=sentences, as_dict=True)

# 使用 signature = tokens, 输入是字典，包含 tokens 和 sequence_len 注意需要padding
def padding(tokens_inp):
    pad_len = max([len(i) for i in tokens_inp])
    return [(i+[""]*pad_len)[:pad_len] for i in tokens_inp]

tokens_list = ["the cat is on the mat".split(" "),"dogs are in the fog".split(" ")]
tokens_list = padding(tokens_list)
tokens_len = [len(i) for i in tokens_list]
input_dict = {
    'tokens': tokens_list,
    'sequence_len' : tokens_len
}
emb_opt_tokens = elmo(inputs=input_dict, signature='tokens', as_dict=True)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 14:04:40.684383 140165026932544 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 14:04:41.996367 140165026932544 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


### 输出
> The output dictionary contains:
>- <u>word_emb</u>: 
>    - the character-based word representations 
>    - `elmo(tokens, as_dict=True)['word_emb']`
>    - *[batch_size, max_length, 512]*
>- <u>lstm_outputs1</u>: 
>    - the first LSTM hidden state 
>    - `elmo(tokens, as_dict=True)['lstm_outputs1']`
>    - *[batch_size, max_length, 1024].*
>- <u>lstm_outputs2</u>: t
>    - he second LSTM hidden state 
>    - `elmo(tokens, as_dict=True)['lstm_outputs2']`
>    - *[batch_size, max_length, 1024].*
>- <u>elmo</u>: 
>    - the weighted sum of the 3 layers, where the weights are trainable. 
>    - `elmo(tokens, as_dict=True)['elmo']`
>    - *[batch_size, max_length, 1024]*
>- <u>default</u>: 
>    - a fixed mean-pooling of all contextualized word representations 
>    - `elmo(tokens, as_dict=True)['default']`
>    - *[batch_size, 1024].*

当输入采用不同方式tokens/sentences时，只有word_emb能确保五种输出结果是一致的；

不一致的原因实际上是因为句子分词后长度不同，需要用空串padding；

如果句子分词后长度都一直，则五种输出结果都一致。

In [132]:
with tf.Session() as sess:
    for output in ['word_emb','lstm_outputs1','lstm_outputs2','elmo','default']:
        sess.run(tf.global_variables_initializer())
        emb = sess.run(emb_opt_sentence[output])
        emb2 = sess.run(emb_opt_tokens[output])
        print(f"signature不同的输入方式(tokens,sentences)，'{output}' 结果是一致的: ",np.all(emb == emb2))

signature不同的输入方式(tokens,sentences)，'word_emb' 结果是一致的:  True
signature不同的输入方式(tokens,sentences)，'lstm_outputs1' 结果是一致的:  False
signature不同的输入方式(tokens,sentences)，'lstm_outputs2' 结果是一致的:  False
signature不同的输入方式(tokens,sentences)，'elmo' 结果是一致的:  False
signature不同的输入方式(tokens,sentences)，'default' 结果是一致的:  False


In [148]:
# 计算好emb
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    emb_tokens = sess.run(emb_opt_tokens['elmo'])
    emb_sentence = sess.run(emb_opt_sentence['elmo'])

In [167]:
print("sentences: ", sentences)
for sen_idx in [0,1]:
    print("\n>>> 使用tokens_list: ", tokens_list[sen_idx])
    consistent = np.all(emb_tokens[sen_idx] == emb_sentence[sen_idx])
    print("两种输入方式下 elmo 的一致性：",consistent)
    if not consistent:
        print("不一致时，tokens和sentence两种方式的tensor值示例：")
        emb_tokens[sen_idx]
        emb_sentence[sen_idx]

sentences:  ['the cat is on the mat', 'dogs are in the fog']

>>> 使用tokens_list:  ['the', 'cat', 'is', 'on', 'the', 'mat']
两种输入方式下 elmo 的一致性： True

>>> 使用tokens_list:  ['dogs', 'are', 'in', 'the', 'fog', '']
两种输入方式下 elmo 的一致性： False
不一致时，tokens和sentence两种方式的tensor值示例：


array([[ 5.4578573e-02, -2.6427570e-01,  4.6843779e-01, ...,
        -1.2844735e-01, -1.8192245e-01,  4.2113167e-01],
       [ 8.0942936e-02,  1.1583884e-01, -1.5670620e-01, ...,
        -1.5915328e-01,  3.8296425e-01, -7.7638179e-02],
       [-7.8964919e-02,  9.4927496e-01, -6.1804903e-01, ...,
        -3.9802760e-01,  2.7152354e-01,  1.0895334e-01],
       [-6.7809001e-02,  9.7187646e-02, -3.6225441e-01, ...,
         1.4475808e-01,  1.7962448e-01, -1.2800075e-02],
       [-2.6190025e-04, -1.2609786e-01, -5.3394711e-01, ...,
         4.9814320e-01,  6.2343001e-01,  5.1778430e-01],
       [ 2.8496963e-01, -7.0748515e-02, -2.2847629e-01, ...,
         1.2616637e-01, -2.6158541e-02,  3.9751023e-01]], dtype=float32)

array([[ 5.4578573e-02, -2.6427570e-01,  4.6843779e-01, ...,
        -1.4077103e-01, -2.6568228e-01,  4.5211977e-01],
       [ 8.0942936e-02,  1.1583884e-01, -1.5670620e-01, ...,
        -2.6896167e-01,  3.3871961e-01,  1.1577174e-02],
       [-7.8964919e-02,  9.4927496e-01, -6.1804903e-01, ...,
        -6.3055909e-01,  3.0943012e-01,  1.5378729e-01],
       [-6.7809001e-02,  9.7187646e-02, -3.6225441e-01, ...,
         7.7499896e-02, -6.1661024e-02,  6.0047179e-02],
       [-2.6190025e-04, -1.2609786e-01, -5.3394711e-01, ...,
         4.2921820e-01,  1.2804650e-01,  5.3191912e-01],
       [-2.8408393e-02, -4.3532155e-02,  4.1301616e-02, ...,
         2.5831670e-02, -1.4298330e-02, -1.6504213e-02]], dtype=float32)